# Modifications ijN1463 model

In [1]:
#Script to modify the model iJN1463.xml for FLYCOP simulation
#07/12/20
#Ana del Ramo Galián
from cobra import Reaction
from cobra import Metabolite

import cobra as cb
import cometspy as c
model=cb.io.read_sbml_model("modelManoli.xml")
model

Academic license - for non-commercial use only - expires 2021-04-19
Using license file /home/iodmc/gurobi.lic


Name,model
Memory address,0x07fa0ca4bf9d0
Number of metabolites,2008
Number of reactions,2625
Number of groups,0
Objective expression,1.0*BiomassKT2440_ME - 1.0*BiomassKT2440_ME_reverse_0e45e
Compartments,"p, c, e"


In [2]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.001203,0,0.00%
cl_e,EX_cl_e,0.001203,0,0.00%
cobalt2_e,EX_cobalt2_e,0.000802,0,0.00%
cu2_e,EX_cu2_e,0.000802,0,0.00%
fe2_e,EX_fe2_e,0.004181,0,0.00%
h_e,EX_h_e,2.337,0,0.00%
k_e,EX_k_e,0.04513,0,0.00%
mg2_e,EX_mg2_e,0.002005,0,0.00%
mobd_e,EX_mobd_e,0.000802,0,0.00%
na1_e,EX_na1_e,0.001003,0,0.00%


In [3]:
C80aPHAtpp_reaction=Reaction('C80aPHAtpp')
C80aPHAtpp_reaction.name='C80aPHA transporter periplasm'
C80aPHAtpp_reaction.lower_bound=0
C80aPHAtpp_reaction.upper_bound=1000
c80apha_p = Metabolite('C80aPHA_p', formula='C8H15O3R', name='C80_Medium_chain_length_aliphatic_Polyhydroxyalkanoate_p',compartment='p')
c80apha_e = Metabolite('C80aPHA_e', formula='C8H15O3R', name='C80_Medium_chain_length_aliphatic_Polyhydroxyalkanoate_e',compartment='e')
C80aPHAtpp_reaction.add_metabolites({model.metabolites.get_by_id('C80aPHA_c'):-1, c80apha_p:1})
C80aPHAtpp_reaction.reaction

'C80aPHA_c --> C80aPHA_p'

In [4]:
C80aPHAtex_reaction=Reaction('C80aPHAtex')
C80aPHAtex_reaction.name='C80aPHA transporter extracellular'
C80aPHAtex_reaction.lower_bound=0
C80aPHAtex_reaction.upper_bound=1000
C80aPHAtex_reaction.add_metabolites({c80apha_p:-1, c80apha_e:1})
C80aPHAtex_reaction.reaction

'C80aPHA_p --> C80aPHA_e'

In [5]:
EX_C80aPHA_e_reaction=Reaction('EX_C80aPHA_e')
EX_C80aPHA_e_reaction.name='C80aPHA exchange'
EX_C80aPHA_e_reaction.lower_bound=0
EX_C80aPHA_e_reaction.upper_bound=1000
EX_C80aPHA_e_reaction.add_metabolites({c80apha_e:-1})
EX_C80aPHA_e_reaction.reaction

'C80aPHA_e --> '

In [6]:
model.reactions.get_by_id('DM_C80aPHA').bounds=(0,0)

### Adding the reactions to the model

In [7]:
model.add_reactions([C80aPHAtex_reaction, C80aPHAtpp_reaction, EX_C80aPHA_e_reaction])

### Quitamos como fuente de carbono la glucosa y el glycol

In [8]:
model.reactions.EX_glc_e.bounds=(0,0)

In [59]:
model.reactions.EX_glycol_e.bounds=(0,0)

In [60]:
model.reactions.EX_o2_e.bounds=(-18.5,0)

## Quitamos las reacciones que ha dicho Juan

In [61]:
for reaction in ['RHACOAE80','PHADPC80','CYSS','NACODA','ALDD2x','ABUTD','ALAR','ALATA_L','FORGLUIH2','ICDHyr','ICL','MALS','MCITL2','OARGDC','ORNCD','PDHbr','PDHcr','PPCSCT','RBK','SUCOAS','UPPN']:
    model.reactions.get_by_id(reaction).bounds=(0,0)

In [62]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0006733,0,0.00%
cl_e,EX_cl_e,0.0006733,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0004488,0,0.00%
cu2_e,EX_cu2_e,0.0004488,0,0.00%
fe2_e,EX_fe2_e,0.00234,0,0.00%
glycol_e,EX_glycol_e,11.64,2,100.00%
k_e,EX_k_e,0.02525,0,0.00%
mg2_e,EX_mg2_e,0.001122,0,0.00%
mobd_e,EX_mobd_e,0.0004488,0,0.00%
na1_e,EX_na1_e,0.0005611,0,0.00%


### Quitamos las reacciones de degradación del glycerol

In [63]:

model.reactions.get_by_id('GLYCOLDHpp').bounds=(0,1000)
model.reactions.get_by_id('EX_tpha_e').bounds=(-5,0)
model.reactions.get_by_id('EX_pet_e').bounds=(0,0)

In [64]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0006733,0,0.00%
cl_e,EX_cl_e,0.0006733,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0004488,0,0.00%
cu2_e,EX_cu2_e,0.0004488,0,0.00%
fe2_e,EX_fe2_e,0.00234,0,0.00%
glycol_e,EX_glycol_e,11.64,2,100.00%
k_e,EX_k_e,0.02525,0,0.00%
mg2_e,EX_mg2_e,0.001122,0,0.00%
mobd_e,EX_mobd_e,0.0004488,0,0.00%
na1_e,EX_na1_e,0.0005611,0,0.00%


In [65]:
comets_model=c.model(model)
comets_model.id='plasticDegradationEstrategy2-dEG'

In [66]:
def ensure_sinks_are_not_exchanges(model):
    rxn_names = ['sink_PHAg']
    for rxn in rxn_names:
        model.reactions.loc[model.reactions.REACTION_NAMES == rxn, 'EXCH'] = False
        model.reactions.loc[model.reactions.REACTION_NAMES == rxn, 'EXCH_IND'] = 0


In [67]:
ensure_sinks_are_not_exchanges(comets_model)

In [68]:

#cb.io.write_sbml_model(model, "plasticDegradationEstrategy2-dEG-GCALDR.xml")
comets_model.write_comets_model()